The homework starts off with my thoughts as I go through the start.  Experiment/functions are towards the end.

In [198]:
import numpy as np
d = 400
sigma = 1.0
r = 12
mu = 0.0

Centers = [[np.random.default_rng().normal(mu,sigma,d)] for i in range(2)]
Centers = np.squeeze(np.asarray(Centers))

for i in range(2):
  Centers = Centers/np.linalg.norm(Centers[i]) * r



Generating points using code from last time.  Main difference is multiplying the normed vectors by 12 so that the ball has a radius of 12.

In [199]:
dist = np.linalg.norm(Centers[0] - Centers[1])
print(dist)
print(12*np.sqrt(2))

17.894081612569032
16.970562748477143


Validating that the centers are roughly 12sqrt(2) distance away from each other 

In [200]:
n = 80
mu1 = Centers[0]
mu2 = Centers[1]

Gaussian1 = [[np.random.default_rng().normal(mu1,sigma,d)] for i in range(n)]
Gaussian2 = [[np.random.default_rng().normal(mu2,sigma,d)] for i in range(n)]
Gaussian1 = np.squeeze(np.asarray(Gaussian1))
Gaussian2 = np.squeeze(np.asarray(Gaussian2))
A = np.concatenate((Gaussian1,Gaussian2),axis=0)
print(A)

[[-0.11236156  1.78802642  0.35796061 ...  0.53118395 -0.89986854
  -0.42414688]
 [-0.76677168 -0.34467065  0.65407114 ... -0.33829524 -1.00964968
   1.14284742]
 [ 0.79764935  0.8472079  -1.5114156  ...  1.99103282 -0.95264174
  -0.46384327]
 ...
 [ 0.64357683 -0.16862523 -1.15835399 ... -0.5283987   0.44253044
  -0.32281664]
 [ 1.37086906 -0.09203618 -0.56750535 ... -0.08253563  0.52055045
   0.78445987]
 [-1.27341732  0.46963824  0.20757174 ... -1.88131677 -0.31866876
  -0.24686589]]


Just generated two gaussians using the centers on the surface of the r=12 d-dimensional ball we generated earlier.  I concatenated them for the problem but going back isnt too hard from here either.  Now we want to choose a basepoint, and calculate the distance from said points to all other points. I'll just choose the first one and be lazy.

In [201]:
basepoint = A[0]
distances = []
for i in range(2*n):
  distances.append(np.linalg.norm(basepoint - A[i]))

Now we have a list of length 160 with all the distances.  We want to know the accuracy of our crude clustering algo.  Where the 80 shortest distances belong to the 1st center, and 80 longer distances belong to the 2nd center.  We already know the 80 first distances belong to the first center.  The problem is that merely sorting and separating doesnt really give feedback on accuracy.

In [202]:
sorted_dist = distances[:]
sorted_dist.sort()
predGaussian1 = sorted_dist[:n]
predGaussian2 = sorted_dist[n+1:]
actualGaussian1 = distances[:n]
actualGaussian2 = distances[n+1:]


Basically after sorting we have a predicted cluster of the first 80 in the sorted list.  We know our real cluster is the first 80 in the unsorted list.  
So our predGaussian1/2 is our sorted distances meaning we are predicting these distances belong to these gaussians.  ActualGaussian1/2 is our unsorted distances from earlier so we know these are the correct.

In [212]:
errors = 0

for x in predGaussian2:
  if (x not in actualGaussian2):  #Im kind of exploiting/relying on the fact that our distances are going to be pretty unique given we calc up to like 10 decimal points.
      errors += 1                  #if a certain distance is in our predicted set but not in our actual set then thats an error.
print(errors)



1


Surprisingly, there are only 1-3 misclassified data points in the crude clustering algorithm with r = 12.  
158/160 = 98.75% accuracy.




Now we start the SVD

This was my first attempt at the SVD code.  My finalized code is located towards the bottom under "SVDCluster" function.  Basically this first attempt at the SVDcluster doesnt really work.

In [88]:
u,d,vt = np.linalg.svd(A)
v = np.transpose(vt)  #this line is a mistake as I learn later on
Vk = [[v[i] for i in range(2)]]
Vk = np.squeeze(np.asarray(Vk))
np.shape(Vk)          #this shape is wrong, the point is to reduce dimensions 


(2, 400)

In [99]:
Ak = A[:]         
for i in range(2*n):
  for j in range(2):
    Ak[i] = (np.dot(A[i],Vk[j])/np.dot(Vk[j],Vk[j])*Vk[j])
np.shape(Ak)
# print(A)
# print(Ak)

(160, 400)

The following code is just the same clustering algo from earlier but using Ak instead of A

In [127]:
basepoint = Ak[0]
distances = []
n = int(len(A)/2)
# print(n)
for i in range(n):
  distances.append(np.linalg.norm(basepoint - A[i]))

sorted_dist = distances[:]
sorted_dist.sort()
predGaussian1 = sorted_dist[:n]
predGaussian2 = sorted_dist[n+1:]
actualGaussian1 = distances[:n]
actualGaussian2 = distances[n+1:]

errors = 0

for x in predGaussian2:
  if (x not in actualGaussian2):
      errors += 1

print(errors)
print("Accuracy is: " + str((n - errors)/(n)))


0
Accuracy is: 1.0


100% accuracy on the first try

:I'm actually going to make some functions to make it even nicer.  

In [408]:
from typing import List
import numpy as np


def generateTwoGaussians(d: int, r: int, n: int) -> List[List[float]]:
  sigma = 1.0
  mu = 0.0
  Centers = [[np.random.default_rng().normal(mu,sigma,d)] for i in range(2)]
  Centers = np.squeeze(np.asarray(Centers))

  for i in range(2):
    Centers = Centers/np.linalg.norm(Centers[i]) * r

  mu1 = Centers[0]
  mu2 = Centers[1]

  Gaussian1 = [[np.random.default_rng().normal(mu1,sigma,d)] for i in range(n)]
  Gaussian2 = [[np.random.default_rng().normal(mu2,sigma,d)] for i in range(n)]
  Gaussian1 = np.squeeze(np.asarray(Gaussian1))
  Gaussian2 = np.squeeze(np.asarray(Gaussian2))
  A = np.concatenate((Gaussian1,Gaussian2),axis=0)
  return A

def crudeCluster(A: List[List[float]]) -> float:
  basepoint = A[0]
  distances = []
  n = int(len(A)/2)
  # print("n is: " + str(n))
  for i in range(2*n):
    distances.append(np.linalg.norm(basepoint - A[i]))
  # print("distances: " + str(distances))
  sorted_dist = distances[:]
  sorted_dist.sort()
  # print("sorted_dist" + str(sorted_dist))
  predGaussian1 = sorted_dist[:n]
  predGaussian2 = sorted_dist[n+1:]
  actualGaussian1 = distances[:n]
  actualGaussian2 = distances[n+1:]
  # print("predGaussian1: " + str(predGaussian1))
  # print("actualGaussian1: " + str(actualGaussian1))
  # print("predGaussian2: " + str(predGaussian2))
  # print("actualGaussian2: " + str(actualGaussian2))

  errors = 0
  for x in predGaussian1:
    if (x not in actualGaussian1):
        errors += 1

  # print("Crude errors:" + str(errors))
  # print("Crude Accuracy is: " + str((n - errors)/(n)))
  return errors

def SVDCluster(A: List[List[float]]) -> float:
  u,d,vt = np.linalg.svd(A)
  # vt = np.transpose(vt)        #we are given v transpose, take transpose again to get v,  I WAS TAKING THE TRANSPOSE OF THE TRANSPOSE BUT IT WORKS SO MUCH BETTER IF I DONT
  Vk = [[vt[i]] for i in range(2)] #take top two right singular vectors
  Vk = np.squeeze(np.asarray(Vk)) #I put this code everywhere because im doing something wrong with the matrix/array notation but this fixes it
  # print(Vk)
  s = (len(A),len(A[0]))        
  Ak = np.zeros(s)            #basically I want to start out with an array of 0s and this is how I did it.
  # print(Ak)
  n = int(len(A)/2)
  for i in range(2*n):
    for j in range(2):
      Ak[i] += (np.dot(A[i],Vk[j])/np.dot(Vk[j],Vk[j]))*Vk[j]
  # print(Ak)
  basepoint = Ak[0]
  distances = []
  
  for i in range(2*n):
    distances.append(np.linalg.norm(basepoint - Ak[i]))

  sorted_dist = distances[:]
  sorted_dist.sort()

  predGaussian1 = sorted_dist[:n]
  predGaussian2 = sorted_dist[n+1:]
  actualGaussian1 = distances[:n]
  actualGaussian2 = distances[n+1:]
  # print("predGaussian1: " + str(predGaussian1))
  # print("actualGaussian1: " + str(actualGaussian1))
  # print("predGaussian2: " + str(predGaussian2))
  # print("actualGaussian2: " + str(actualGaussian2))

  errors = 0
  for x in predGaussian1:
    if (x not in actualGaussian1):
        errors += 1

  # print("SVD Cluster errors:" + str(errors))
  # print("SVD Accuracy is: " + str((n - errors)/(n)))
  return errors


In [406]:
B = generateTwoGaussians(d = 400,r = 12,n = 80)
crudeCluster(B)
SVDCluster(B)

Crude errors:1
Crude Accuracy is: 0.9875
SVD Cluster errors:0
SVD Accuracy is: 1.0


0

In [409]:
CrudeErrors = 0
SVDErrors = 0
n = 50*80
for i in range(50):
  C = generateTwoGaussians(d=400, r = 12, n = 80)
  CrudeErrors += crudeCluster(C)
  SVDErrors += SVDCluster(C)

print("d = 400, r = 12, 80 pts per gaussian center")
print("Crude errors:" + str(CrudeErrors))
print("Crude Accuracy is: " + str((n - CrudeErrors)/(n)))
print("SVD Cluster errors:" + str(SVDErrors))
print("SVD Accuracy is: " + str((n - SVDErrors)/(n)))

d = 400, r = 12, 80 pts per gaussian center
Crude errors:20
Crude Accuracy is: 0.995
SVD Cluster errors:0
SVD Accuracy is: 1.0


In [410]:
CrudeErrors = 0
SVDErrors = 0
n = 50*80
for i in range(50):
  C = generateTwoGaussians(d=400, r = 8, n = 80)
  CrudeErrors += crudeCluster(C)
  SVDErrors += SVDCluster(C)

print("d = 400, r = 8, 80 pts per gaussian center")
print("Crude errors:" + str(CrudeErrors))
print("Crude Accuracy is: " + str((n - CrudeErrors)/(n)))
print("SVD Cluster errors:" + str(SVDErrors))
print("SVD Accuracy is: " + str((n - SVDErrors)/(n)))

d = 400, r = 8, 80 pts per gaussian center
Crude errors:452
Crude Accuracy is: 0.887
SVD Cluster errors:0
SVD Accuracy is: 1.0


In [411]:
CrudeErrors = 0
SVDErrors = 0
n = 50*80
for i in range(50):
  C = generateTwoGaussians(d=400, r = 4.2, n = 80)
  CrudeErrors += crudeCluster(C)
  SVDErrors += SVDCluster(C)

print("d = 400, r = 4.2, 80 pts per gaussian center")
print("Crude errors:" + str(CrudeErrors))
print("Crude Accuracy is: " + str((n - CrudeErrors)/(n)))
print("SVD Cluster errors:" + str(SVDErrors))
print("SVD Accuracy is: " + str((n - SVDErrors)/(n)))

d = 400, r = 4.2, 80 pts per gaussian center
Crude errors:1356
Crude Accuracy is: 0.661
SVD Cluster errors:128
SVD Accuracy is: 0.968


In [413]:
CrudeErrors = 0
SVDErrors = 0
n = 50*80
for i in range(50):
  C = generateTwoGaussians(d=6, r = 8, n = 80)
  CrudeErrors += crudeCluster(C)
  SVDErrors += SVDCluster(C)

print("d = 6, r = 8, 80 pts per gaussian center")
print("Crude errors:" + str(CrudeErrors))
print("Crude Accuracy is: " + str((n - CrudeErrors)/(n)))
print("SVD Cluster errors:" + str(SVDErrors))
print("SVD Accuracy is: " + str((n - SVDErrors)/(n)))

d = 6, r = 8, 80 pts per gaussian center
Crude errors:10
Crude Accuracy is: 0.9975
SVD Cluster errors:1
SVD Accuracy is: 0.99975


SVD might be too good... I might be cheating

I was cheating a little.  But I fixed it.

Thoughts uhh pretty interesting.  I'm still a little bit worried that I didn't implement the SVD correctly, but the fact that it performs poorly on r = 4.2 distance gaussians gives me hope. If I am doing it correctly, then its pretty incredible how well it works.

# In this homework, you'll implement the strategy for separating Gaussians described in 3.9.3 "Clustering a Mixture of Spherical Gaussians" (and also relevant is 2.8 "Separating Gaussians").  The idea is that a crude clustering algorithm plus SVD yields a better clustering algorithm.

Generating Data Points

Working in 400-dimensional space, first generate 2 random points 
μ
1
 and 
μ
2
 on the surface of the sphere centered at the origin with radius r=12.  (Note that in 400-dimensional space, we expect the two vectors two be nearly orthogonal, so the distance between the two points should be around 
12
2
.  You might want to check that this is so for your two points.)  These will be the centers of our Gaussian distributions.  Generate 80 random points from a Gaussian distribution with center 
μ
1
, and then another 80 points from a Gaussian distribution with center 
μ
2
.  For all Gaussian distrubutions, we'll just use 
σ
=
1
.

The Crude Clustering Algorithm

The crude algorithm assumes in advance that there are the same number of points (80 of them) generated from each Gaussian, but doesn't know which is which. 

Pick one data point as a basepoint.  You could pick randomly, or just always use the first data point generated.  Or, you could slightly reduce the chances to choose a weird outlier as your basepoint as follows:  Compute the distances between all pairs of points from your set of 160 and find the minimum.  Choose as your basepoint one of the points that has minimum distance to another point.
Make a list of the distances from your basepoint to all 160 points in the set. 
Take the 80 points (including the basepoint) whose distances to the basepoint are smallest, and say that these are probably from the same distribution as your basepoint.
You should find that this crude method works pretty well if you generate data as described above, but works less will if you decrease r to 8 or 4. 

Optional: Perhaps Somewhat Less Crude Clustering Algorithm

Instead of assuming there are 80 points from each distribution, you could in step 3 use a better method that looks for a cluster of the smallest points in your list, looking for a gap between the small values and the large values.  We haven't discussed such methods in this class, but maybe you know one or would like to find one some other way. 

If you use another method, your method should only use the list of distances from the basepoint, and not any other information about the positions of the data points.  Of course there are many methods of clustering the original data set, but if you use some other method, it probably sort of defeats the purpose of this exercise.  If you do use a method that doesn't assume the same number of points from each Gaussian, then instead of generating 80 points from each Gaussian, you could randomly generate 60-100 points from each.

Clustering Using SVD

Find the best fit 2-dimensional subspace K to your 160 data points.  To do this, use the SVD and take the top 2 right singular vectors as a basis for K.  (Don't write your own code to find the singular vectors.  Let Matlab or numpy find the SVD for you.)
Compute the orthogonal projections of your data points onto K.
Use your crude clustering algorithm on the set of projections.
The Experiment (or at least "playing around to see how well these work")

Generate data as described in "Generating Data Points".  Use the Crude Clustering Algorithm to try to separate the points.  Compute what percentage of the points were classified correctly (or do the percentage classified incorrectly, if you prefer that point of view).  Then try the separation via the Clustering Using SVD algorithm, and again compute the percentage classified correctly.  Repeat 50 times and report the average percentages over the 50 repetitions.

Then do it all again, using r=8 and r=4.2 instead of r=12.  (Recall r is the distance from the origin to each Gaussian center.)  Also try it using r=8 again, but in 6-dimensional space instead of d=400.

Comment on the results.   Submit: (1) your code (in a text file or some format that someone could actually try to run---not a pdf of a screenshot), (2) some sample output, and (3) your comments.

Bonus Complication

If you want extra points, you can do this all with 4 gaussian distributions instead of just 2.  You'd have a set of 320 points instead of 160. 

The crude algorithm would choose a basepoint and then select the 80 nearest points for one cluster.  Remove these from the list, choose a new basepoint, and again select the 80 nearest points for the next cluster.  Remove these from the list, and then repeat again to separate the remaining 160 points.
The SVD algorithm would use the best-fit 4-dimensional subspace instead of the best-fit 2-dimensional subspace. 
 